In [1]:
# importons les librairies dont nous avons besoin.

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn-muted')
%matplotlib inline
import math
from sklearn import decomposition
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [2]:
data = pd.read_csv("clean_all_data.csv")
data = data.drop(columns=["TAIL_NUM", "FL_NUM", "ORIGIN_AIRPORT_ID", "DEST_AIRPORT_ID"])
save = data

In [3]:
data.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,AIRLINE_ID,DEP_TIME,ARR_TIME,ARR_DELAY_NEW,AIR_TIME,DEP_0_280,ARR_0_220,AIR_0_360,DEP_0_280_TIME,ARR_0_220_TIME,AIR_0_360_TIME
0,1,6,3,19805,657,872,0.0,132.0,0,0,1,657,872,0.0
1,1,7,4,19805,656,866,0.0,126.0,0,0,1,656,866,0.0
2,1,8,5,19805,655,885,7.0,135.0,0,0,1,655,885,0.0
3,1,9,6,19805,662,873,0.0,129.0,0,0,1,662,873,0.0
4,1,10,7,19805,760,991,113.0,137.0,0,0,1,760,991,0.0


Du fait du grand nombre de données il est impossible de réutiliser la méthode des dummies sur l'ensemble de notre dataset ni d'utiliser de régression polynomiale de degré 3. Nous allons donc comparer 2 approches :

- Une régression linéaire avec mean encoding sur l'ensemble du dataset
- Une régression polynomiale de degrée 2 avec mean encoding sur l'ensemble du dataset

Ces deux valeurs seront celles que nous utiliserons comme baseline error dans notre notebook final.

In [4]:
xtrain, xtest, ytrain, ytest = train_test_split(data.loc[:, data.columns != 'ARR_DELAY_NEW'], data[["ARR_DELAY_NEW"]], test_size=0.2, random_state=42)
train = pd.merge(xtrain, ytrain, right_index=True, left_index=True)
columns = data.loc[:, "MONTH":"DAY_OF_WEEK"].columns
for column in columns:
    items = train.groupby(column).mean()["ARR_DELAY_NEW"].to_dict()
    for key, value in items.items():
        train.loc[train[column] == key, column] = value
        xtest.loc[xtest[column] == key, column] = value
xtrain = train.loc[:, train.columns != "ARR_DELAY_NEW"]
ytrain = train[["ARR_DELAY_NEW"]]

In [5]:
test = pd.merge(xtest, ytest, right_index=True, left_index=True)
pd.concat([test, train], ignore_index=True).to_csv("mean_encoding_data.csv", columns=data.columns, encoding="utf-8", index=False)

In [6]:
regr = LinearRegression()
regr.fit(xtrain, ytrain)
ypred = regr.predict(xtest)
print("Mean squared error: %.2f"% math.sqrt(mean_squared_error(ytest, ypred)))

Mean squared error: 36.50


In [7]:
model = make_pipeline(PolynomialFeatures(2), linear_model.Ridge())
model.fit(xtrain, ytrain)
math.sqrt(np.mean((model.predict(xtest) - ytest) ** 2).values[0])

36.30942244734323

Nous utiliserons donc 36.5 et 36.3 comme valeurs de référence. Reste à voir s'il nous est possible d'obtenir de meilleurs résultats en utilisant un algorithme par compagnie aérienne.